In [29]:
import pandas as pd #Pandas 모듈 불러오기
import numpy as np
import openpyxl
from datetime import datetime
import matplotlib.pyplot as plt
import scipy.stats as stats
from tqdm import tqdm
tqdm.pandas()
plt.rc('font', family='Malgun Gothic')

## 1) 순환업종 현황분석

In [ ]:
df_cycle = pd.read_csv('../Data/순환분석.csv',encoding='utf-8')
df_cycle.head()

In [40]:
df_cycle['날짜'] = pd.to_datetime(df_cycle['날짜'],format="%Y-%m-%d")

In [42]:
df_cycle['월'] = df_cycle['날짜'].dt.month.astype(str) +'월'
df_cycle[['연도','월','가맹점 골목상권분류','가맹점 대분류','사용금액']].to_excel('../Result/800. 30억 규제 영향 분석/5.순환데이터 시각화.xlsx',index=False)

ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ

In [43]:
def outlier(x, target_column):
    q1 = np.quantile(x[target_column], 0.25)
    q3 = np.quantile(x[target_column], 0.75)
    iqr = q3-q1

    x.loc[((q1 - iqr*1.5) < x[target_column]) & (x[target_column] < (q3 + iqr*1.5)), f'이상치_{target_column}'] = '정상'
    return x

In [45]:
#na값 제거
df_cycle = df_cycle[~df_cycle['가맹점 골목상권분류'].isna()]

In [ ]:
df_cycle['날짜'] = pd.to_datetime(df_cycle['날짜'])
df_cycle['연월'] = df_cycle['날짜'].dt.year.astype(str) + "/" + df_cycle['날짜'].dt.month.astype(str)
df_cycle

골목상권분류별 규제 전/후 총 매출액 평균 차이 T-Test

In [48]:
for store in df_cycle['가맹점 골목상권분류'].unique():
    print(store)
    
    tmp_df = df_cycle.loc[(df_cycle['가맹점 골목상권분류'] == store)]
    grp_side_store = tmp_df.groupby(['연월', '연령대', '가맹점 골목상권분류', '가맹점 가맹점 주소', '가맹점 대분류', '가맹점 사업자 번호']).agg({'사용금액':'sum'}).reset_index()
    before_side_store = grp_side_store[grp_side_store['연월'] == '2023/6'].groupby('가맹점 사업자 번호')['사용금액'].sum().reset_index()
    after_side_store = grp_side_store[grp_side_store['연월'] == '2023/7'].groupby('가맹점 사업자 번호')['사용금액'].sum().reset_index()
    
    before_side_store = outlier(before_side_store, '사용금액')
    after_side_store = outlier(after_side_store, '사용금액')
    out_store = pd.merge(before_side_store,after_side_store,how='inner',on='가맹점 사업자 번호')
    out_store = out_store.loc[(out_store['이상치_사용금액_x']=='정상') & (out_store['이상치_사용금액_y']=='정상')].drop(columns=['이상치_사용금액_x','이상치_사용금액_y'])
    out_store.rename(columns={'사용금액_x':'before_restrict', '사용금액_y':'after_restrict'},inplace=True)
    
    t_statistic, p_value = stats.ttest_rel(out_store['before_restrict'],out_store['after_restrict'])
    
    
    print("t-statistic:", t_statistic)
    print("p-value:", p_value)

    alpha = 0.05
    if p_value < alpha:
        print("귀무가설을 기각합니다. 두 샘플의 평균은 통계적으로 유의미하게 다릅니다.")
    else:
        print("귀무가설을 기각하지 않습니다. 두 샘플의 평균은 통계적으로 차이가 없습니다.")
        
    print("규제 전 사용금액 총합: " + str(out_store['before_restrict'].sum()))
    print("규제 후 사용금액 총합: " + str(out_store['after_restrict'].sum()))
    
    print((out_store['after_restrict'].sum()-out_store['before_restrict'].sum())/out_store['before_restrict'].sum()*100)
    
    #out_store['증감율'] = (out_store['after_restrict']-out_store['before_restrict'])/out_store['before_restrict']*100
    #out_store = out_store.sort_values(by='증감율',ascending=False)
    print('----------------------------------\n\n')

기타
t-statistic: -3.3279374830627235
p-value: 0.0009749470165018397
귀무가설을 기각합니다. 두 샘플의 평균은 통계적으로 유의미하게 다릅니다.
규제 전 사용금액 총합: 75077267
규제 후 사용금액 총합: 85683870
14.127582720878745
----------------------------------


골목상권
t-statistic: -4.373091729620018
p-value: 1.4586803288227427e-05
귀무가설을 기각합니다. 두 샘플의 평균은 통계적으로 유의미하게 다릅니다.
규제 전 사용금액 총합: 111767529
규제 후 사용금액 총합: 128926454
15.352334576529824
----------------------------------


대형
t-statistic: 0.788913693253103
p-value: 0.4329452457410591
귀무가설을 기각하지 않습니다. 두 샘플의 평균은 통계적으로 차이가 없습니다.
규제 전 사용금액 총합: 42536128
규제 후 사용금액 총합: 39446831
-7.2627602587616815
----------------------------------




가맹점 대분류별 규제 전/후 총 매출액 평균 차이 T-Test

In [49]:
for category in df_cycle.loc[(df_cycle['연월']=='2023/6') | (df_cycle['연월']=='2023/7'),'가맹점 대분류'].unique():
    print(category)
    
    tmp_df = df_cycle.loc[(df_cycle['가맹점 대분류'] == category)]
    grp_side_store = tmp_df.groupby(['연월', '연령대', '가맹점 골목상권분류', '가맹점 가맹점 주소', '가맹점 대분류', '가맹점 사업자 번호']).agg({'사용금액':'sum'}).reset_index()
    before_side_store = grp_side_store[(grp_side_store['연월'] == '2023/6')].groupby('가맹점 사업자 번호')['사용금액'].sum().reset_index()
    after_side_store = grp_side_store[(grp_side_store['연월'] == '2023/7')].groupby('가맹점 사업자 번호')['사용금액'].sum().reset_index()
    
    before_side_store = outlier(before_side_store, '사용금액')
    after_side_store = outlier(after_side_store, '사용금액')
    out_store = pd.merge(before_side_store,after_side_store,how='inner',on='가맹점 사업자 번호')
    out_store = out_store.loc[(out_store['이상치_사용금액_x']=='정상') & (out_store['이상치_사용금액_y']=='정상')].drop(columns=['이상치_사용금액_x','이상치_사용금액_y'])
    out_store.rename(columns={'사용금액_x':'before_restrict', '사용금액_y':'after_restrict'},inplace=True)
    
    t_statistic, p_value = stats.ttest_rel(out_store['before_restrict'],out_store['after_restrict'])
    
    
    print("t-statistic:", t_statistic)
    print("p-value:", p_value)

    alpha = 0.05
    if p_value < alpha:
        print("귀무가설을 기각합니다. 두 샘플의 평균은 통계적으로 유의미하게 다릅니다.")
    else:
        print("귀무가설을 기각하지 않습니다. 두 샘플의 평균은 통계적으로 차이가 없습니다.")
    
    print("규제 전 사용금액 총합: " + str(out_store['before_restrict'].sum()))
    print("규제 후 사용금액 총합: " + str(out_store['after_restrict'].sum()))
    print((out_store['after_restrict'].sum()-out_store['before_restrict'].sum())/out_store['before_restrict'].sum()*100)

    #out_store['증감율'] = (out_store['after_restrict']-out_store['before_restrict'])/out_store['before_restrict']*100
    #out_store = out_store.sort_values(by='증감율',ascending=False)
    print('----------------------------------\n\n')

소매
t-statistic: -1.1264465036195008
p-value: 0.26186336006914507
귀무가설을 기각하지 않습니다. 두 샘플의 평균은 통계적으로 차이가 없습니다.
규제 전 사용금액 총합: 35106519
규제 후 사용금액 총합: 37542330
6.938343844344123
----------------------------------


생활서비스
t-statistic: -2.4426233689931536
p-value: 0.017184554630681262
귀무가설을 기각합니다. 두 샘플의 평균은 통계적으로 유의미하게 다릅니다.
규제 전 사용금액 총합: 5620560
규제 후 사용금액 총합: 7224980
28.545554179654697
----------------------------------


기타
t-statistic: -1.3933395251918255
p-value: 0.16486804909850714
귀무가설을 기각하지 않습니다. 두 샘플의 평균은 통계적으로 차이가 없습니다.
규제 전 사용금액 총합: 47666885
규제 후 사용금액 총합: 51111621
7.226685779865834
----------------------------------


자동차
t-statistic: 1.038144718639254
p-value: 0.30487429456529946
귀무가설을 기각하지 않습니다. 두 샘플의 평균은 통계적으로 차이가 없습니다.
규제 전 사용금액 총합: 27755648
규제 후 사용금액 총합: 24631446
-11.256094615409447
----------------------------------


음식
t-statistic: -3.7105851044202574
p-value: 0.00024033763209715925
귀무가설을 기각합니다. 두 샘플의 평균은 통계적으로 유의미하게 다릅니다.
규제 전 사용금액 총합: 76396355
규제 후 사용금액 총합: 88109882
15.3325

C:\Users\addmin\AppData\Local\Temp\ipykernel_1344\1333282865.py:29: RuntimeWarning: invalid value encountered in scalar divide
  print((out_store['after_restrict'].sum()-out_store['before_restrict'].sum())/out_store['before_restrict'].sum()*100)


## 2) 골목상권 현황분석

In [ ]:
data_가맹 = pd.read_csv('../Data/Preprocessed/사용자분석.csv', encoding='utf-8')
data_가맹.head()

In [51]:
#전처리1. 부여군만 필터링
data_가맹_부여 = data_가맹.loc[data_가맹['시']=='부여군']

In [ ]:
#사용자코드별 사용금액 합산
grp_out =  data_가맹_부여.groupby(['사용자코드','연령대'])['사용금액'].sum().reset_index()
grp_out.head(5)

In [53]:
#이상치 제거 함수 정의
def outlier(x, target_column):
    q1 = x[target_column].quantile(0.25)
    q3 = x[target_column].quantile(0.75)
    iqr = q3-q1

    x.loc[((q1 - iqr*1.5) < x[target_column]) & (x[target_column] < (q3 + iqr*1.5)), f'이상치_{target_column}'] = '정상'
    return x

In [54]:
#함수 한 행씩 apply 하기
no_outlier_grp = grp_out.groupby('연령대').progress_apply(lambda x : outlier(x, '사용금액')).reset_index(drop = True)

100%|██████████| 7/7 [00:00<00:00, 145.53it/s]


In [55]:
##이상치 제거한 변수 정의
data_가맹_부여_이상치제거 = data_가맹_부여.loc[data_가맹_부여['사용자코드'].isin(no_outlier_grp.loc[no_outlier_grp['이상치_사용금액'] == '정상','사용자코드'])]

In [56]:
#전처리2. 대분류 기타 제거
data_가맹_부여_기타제거 = data_가맹_부여_이상치제거.loc[data_가맹_부여_이상치제거['대분류']!='기타' ]

In [57]:
#전처리3. 대분류 + _//_ + 소분류 구분 파생변수 생성
data_가맹_부여_기타제거['소분류_pp'] = data_가맹_부여_기타제거['대분류'] + "_//_"+ data_가맹_부여_기타제거['소분류']

C:\Users\addmin\AppData\Local\Temp\ipykernel_1344\3448087450.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_가맹_부여_기타제거['소분류_pp'] = data_가맹_부여_기타제거['대분류'] + "_//_"+ data_가맹_부여_기타제거['소분류']


In [58]:
#분석행정도 4개 권역 정의
분석행정동 = ['부여읍','규암면','홍산면']

In [59]:
# na값과 분석행정동이 아닌 행정동 기타로 지정
data_가맹_부여_기타제거.loc[(~data_가맹_부여_기타제거['읍/면'].isin(분석행정동)) & (~data_가맹_부여_기타제거['읍/면'].isna()), '읍/면'] = '기타'
data_가맹_부여_기타제거.loc[(~data_가맹_부여_기타제거['가맹점 주소'].isin(분석행정동)) & (~data_가맹_부여_기타제거['가맹점 주소'].isna()), '가맹점 주소'] = '기타'

시각화 데이터 추출

In [62]:
data_가맹_부여_기타제거.groupby(['연도','월','연령대','읍/면','골목상권분류','대분류','가맹점 주소']).agg({'사용금액':'sum'}).reset_index().to_excel('../Result/800. 30억 규제 영향 분석/2.골목상권 EDA.xlsx',index=False)

### 골목상권 분류별 T-Test

In [64]:
for store in data_가맹_부여_기타제거['골목상권분류'].unique():
    print(store)
    
    tmp_df = data_가맹_부여_기타제거.loc[(data_가맹_부여_기타제거['골목상권분류'] == store)]
    grp_side_store = tmp_df.groupby(['연도','월','연령대','읍/면','소분류_pp','가맹점 주소','가맹점 사업자 번호']).agg({'사용금액':'sum', '사용자코드':'nunique'}).reset_index()
    before_side_store = grp_side_store[(grp_side_store['연도'] == '2023년') & (grp_side_store['월'] == '6월')].groupby('가맹점 사업자 번호')['사용금액'].sum().reset_index()
    after_side_store = grp_side_store[(grp_side_store['연도'] == '2023년') & (grp_side_store['월'] == '7월')].groupby('가맹점 사업자 번호')['사용금액'].sum().reset_index()
    
    before_side_store = outlier(before_side_store, '사용금액')
    after_side_store = outlier(after_side_store, '사용금액')
    out_store = pd.merge(before_side_store,after_side_store,how='inner',on='가맹점 사업자 번호')
    out_store = out_store.loc[(out_store['이상치_사용금액_x']=='정상') & (out_store['이상치_사용금액_y']=='정상')].drop(columns=['이상치_사용금액_x','이상치_사용금액_y'])
    out_store.rename(columns={'사용금액_x':'before_restrict', '사용금액_y':'after_restrict'},inplace=True)
    
    t_statistic, p_value = stats.ttest_rel(out_store['before_restrict'],out_store['after_restrict'])
    
    
    print("t-statistic:", t_statistic)
    print("p-value:", p_value)

    alpha = 0.05
    if p_value < alpha:
        print("귀무가설을 기각합니다. 두 샘플의 평균은 통계적으로 유의미하게 다릅니다.")
    else:
        print("귀무가설을 기각하지 않습니다. 두 샘플의 평균은 통계적으로 차이가 없습니다.")
        
    print("규제 전 사용금액 총합: " + str(out_store['before_restrict'].sum()))
    print("규제 후 사용금액 총합: " + str(out_store['after_restrict'].sum()))
    
    print((out_store['after_restrict'].sum()-out_store['before_restrict'].sum())/out_store['before_restrict'].sum()*100)
    
    #out_store['증감율'] = (out_store['after_restrict']-out_store['before_restrict'])/out_store['before_restrict']*100
    #out_store = out_store.sort_values(by='증감율',ascending=False)
    print('----------------------------------\n\n')

골목상권
t-statistic: -12.377235033937291
p-value: 1.230048939976639e-32
귀무가설을 기각합니다. 두 샘플의 평균은 통계적으로 유의미하게 다릅니다.
규제 전 사용금액 총합: 756155645
규제 후 사용금액 총합: 942654524
24.664086055986527
----------------------------------


대형
t-statistic: 2.582271403455665
p-value: 0.010819861267396773
귀무가설을 기각합니다. 두 샘플의 평균은 통계적으로 유의미하게 다릅니다.
규제 전 사용금액 총합: 430422861
규제 후 사용금액 총합: 303950993
-29.38316698749884
----------------------------------


기타
t-statistic: -4.995830368517176
p-value: 1.6730565043933705e-06
귀무가설을 기각합니다. 두 샘플의 평균은 통계적으로 유의미하게 다릅니다.
규제 전 사용금액 총합: 285387213
규제 후 사용금액 총합: 342966123
20.175714740239606
----------------------------------




### 골목상권의 가맹점 주소별 T-Test

In [65]:
for place in data_가맹_부여_기타제거['가맹점 주소'].unique():
    print(place)
    
    tmp_df = data_가맹_부여_기타제거.loc[(data_가맹_부여_기타제거['가맹점 주소'] == place)]
    grp_side_store = tmp_df.groupby(['연도','월','연령대','읍/면','골목상권분류','소분류_pp','가맹점 주소','가맹점 사업자 번호']).agg({'사용금액':'sum', '사용자코드':'nunique'}).reset_index()
    before_side_store = grp_side_store[(grp_side_store['연도'] == '2023년') & (grp_side_store['월'] == '6월') & (grp_side_store['골목상권분류']=='골목상권')].groupby('가맹점 사업자 번호')['사용금액'].sum().reset_index()
    after_side_store = grp_side_store[(grp_side_store['연도'] == '2023년') & (grp_side_store['월'] == '7월') & (grp_side_store['골목상권분류']=='골목상권')].groupby('가맹점 사업자 번호')['사용금액'].sum().reset_index()
    
    before_side_store = outlier(before_side_store, '사용금액')
    after_side_store = outlier(after_side_store, '사용금액')
    out_store = pd.merge(before_side_store,after_side_store,how='inner',on='가맹점 사업자 번호')
    out_store = out_store.loc[(out_store['이상치_사용금액_x']=='정상') & (out_store['이상치_사용금액_y']=='정상')].drop(columns=['이상치_사용금액_x','이상치_사용금액_y'])
    out_store.rename(columns={'사용금액_x':'before_restrict', '사용금액_y':'after_restrict'},inplace=True)
    
    t_statistic, p_value = stats.ttest_rel(out_store['before_restrict'],out_store['after_restrict'])
    
    
    print("t-statistic:", t_statistic)
    print("p-value:", p_value)

    alpha = 0.05
    if p_value < alpha:
        print("귀무가설을 기각합니다. 두 샘플의 평균은 통계적으로 유의미하게 다릅니다.")
    else:
        print("귀무가설을 기각하지 않습니다. 두 샘플의 평균은 통계적으로 차이가 없습니다.")
        
    print("규제 전 사용금액 총합: " + str(out_store['before_restrict'].sum()))
    print("규제 후 사용금액 총합: " + str(out_store['after_restrict'].sum()))
    print((out_store['after_restrict'].sum()-out_store['before_restrict'].sum())/out_store['before_restrict'].sum()*100)

    #out_store['증감율'] = (out_store['after_restrict']-out_store['before_restrict'])/out_store['before_restrict']*100
    #out_store = out_store.sort_values(by='증감율',ascending=False)
    print('----------------------------------\n\n')

부여읍
t-statistic: -12.027780380852679
p-value: 5.4528661056728794e-30
귀무가설을 기각합니다. 두 샘플의 평균은 통계적으로 유의미하게 다릅니다.
규제 전 사용금액 총합: 508951292
규제 후 사용금액 총합: 663361303
30.33885824186099
----------------------------------


규암면
t-statistic: -2.462666027717797
p-value: 0.015761664857378694
귀무가설을 기각합니다. 두 샘플의 평균은 통계적으로 유의미하게 다릅니다.
규제 전 사용금액 총합: 89671500
규제 후 사용금액 총합: 106502309
18.76940722526109
----------------------------------


기타
t-statistic: -0.6433440748979316
p-value: 0.520888605899898
귀무가설을 기각하지 않습니다. 두 샘플의 평균은 통계적으로 차이가 없습니다.
규제 전 사용금액 총합: 109300890
규제 후 사용금액 총합: 113131192
3.5043648775412537
----------------------------------


홍산면
t-statistic: -3.143204504497048
p-value: 0.0025800412975832353
귀무가설을 기각합니다. 두 샘플의 평균은 통계적으로 유의미하게 다릅니다.
규제 전 사용금액 총합: 60015373
규제 후 사용금액 총합: 72121870
20.172326513741737
----------------------------------




### 골목상권의 업종 대분류별 T-Test

In [66]:
for category in data_가맹_부여_기타제거.loc[(data_가맹_부여_기타제거['연도']=='2023년') & ((data_가맹_부여_기타제거['월']=='6월') | (data_가맹_부여_기타제거['월']=='7월')) & (data_가맹_부여_기타제거['골목상권분류'] == '골목상권'),'대분류'].unique():
    print(category)
    
    tmp_df = data_가맹_부여_기타제거.loc[(data_가맹_부여_기타제거['대분류'] == category)]
    grp_side_store = tmp_df.groupby(['연도','월','연령대','읍/면','골목상권분류','가맹점 주소','가맹점 사업자 번호']).agg({'사용금액':'sum', '사용자코드':'nunique'}).reset_index()
    before_side_store = grp_side_store[(grp_side_store['연도'] == '2023년') & (grp_side_store['월'] == '6월') & (grp_side_store['골목상권분류']=='골목상권')].groupby('가맹점 사업자 번호')['사용금액'].sum().reset_index()
    after_side_store = grp_side_store[(grp_side_store['연도'] == '2023년') & (grp_side_store['월'] == '7월') & (grp_side_store['골목상권분류']=='골목상권')].groupby('가맹점 사업자 번호')['사용금액'].sum().reset_index()
    
    before_side_store = outlier(before_side_store, '사용금액')
    after_side_store = outlier(after_side_store, '사용금액')
    out_store = pd.merge(before_side_store,after_side_store,how='inner',on='가맹점 사업자 번호')
    out_store = out_store.loc[(out_store['이상치_사용금액_x']=='정상') & (out_store['이상치_사용금액_y']=='정상')].drop(columns=['이상치_사용금액_x','이상치_사용금액_y'])
    out_store.rename(columns={'사용금액_x':'before_restrict', '사용금액_y':'after_restrict'},inplace=True)
    
    t_statistic, p_value = stats.ttest_rel(out_store['before_restrict'],out_store['after_restrict'])
    
    
    print("t-statistic:", t_statistic)
    print("p-value:", p_value)

    alpha = 0.05
    if p_value < alpha:
        print("귀무가설을 기각합니다. 두 샘플의 평균은 통계적으로 유의미하게 다릅니다.")
    else:
        print("귀무가설을 기각하지 않습니다. 두 샘플의 평균은 통계적으로 차이가 없습니다.")
    
    print("규제 전 사용금액 총합: " + str(out_store['before_restrict'].sum()))
    print("규제 후 사용금액 총합: " + str(out_store['after_restrict'].sum()))
    print((out_store['after_restrict'].sum()-out_store['before_restrict'].sum())/out_store['before_restrict'].sum()*100)

    #out_store['증감율'] = (out_store['after_restrict']-out_store['before_restrict'])/out_store['before_restrict']*100
    #out_store = out_store.sort_values(by='증감율',ascending=False)
    print('----------------------------------\n\n')

음식
t-statistic: -10.134503798667048
p-value: 3.276369742530848e-22
귀무가설을 기각합니다. 두 샘플의 평균은 통계적으로 유의미하게 다릅니다.
규제 전 사용금액 총합: 501279791
규제 후 사용금액 총합: 626098791
24.900066238656727
----------------------------------


소매
t-statistic: -6.391030091119976
p-value: 7.963278348876835e-10
귀무가설을 기각합니다. 두 샘플의 평균은 통계적으로 유의미하게 다릅니다.
규제 전 사용금액 총합: 221133944
규제 후 사용금액 총합: 278205002
25.808366172856758
----------------------------------


생활서비스
t-statistic: -3.5896958756604738
p-value: 0.0004752145681750816
귀무가설을 기각합니다. 두 샘플의 평균은 통계적으로 유의미하게 다릅니다.
규제 전 사용금액 총합: 57792100
규제 후 사용금액 총합: 68411399
18.375001081462692
----------------------------------




### 골목상권의 업종 소분류별 T-Test

In [67]:
for category in data_가맹_부여_기타제거.loc[(data_가맹_부여_기타제거['연도']=='2023년') & ((data_가맹_부여_기타제거['월']=='6월') | (data_가맹_부여_기타제거['월']=='7월')) & (data_가맹_부여_기타제거['골목상권분류'] == '골목상권') & (data_가맹_부여_기타제거['대분류'] == '음식'),'소분류_pp'].unique():
    print(category)
    
    tmp_df = data_가맹_부여_기타제거.loc[(data_가맹_부여_기타제거['소분류_pp'] == category)]
    grp_side_store = tmp_df.groupby(['연도','월','연령대','읍/면','골목상권분류','가맹점 주소','가맹점 사업자 번호']).agg({'사용금액':'sum', '사용자코드':'nunique'}).reset_index()
    before_side_store = grp_side_store[(grp_side_store['연도'] == '2023년') & (grp_side_store['월'] == '6월') & (grp_side_store['골목상권분류']=='골목상권')].groupby('가맹점 사업자 번호')['사용금액'].sum().reset_index()
    after_side_store = grp_side_store[(grp_side_store['연도'] == '2023년') & (grp_side_store['월'] == '7월') & (grp_side_store['골목상권분류']=='골목상권')].groupby('가맹점 사업자 번호')['사용금액'].sum().reset_index()
    
    before_side_store = outlier(before_side_store, '사용금액')
    after_side_store = outlier(after_side_store, '사용금액')
    out_store = pd.merge(before_side_store,after_side_store,how='inner',on='가맹점 사업자 번호')
    out_store = out_store.loc[(out_store['이상치_사용금액_x']=='정상') & (out_store['이상치_사용금액_y']=='정상')].drop(columns=['이상치_사용금액_x','이상치_사용금액_y'])
    out_store.rename(columns={'사용금액_x':'before_restrict', '사용금액_y':'after_restrict'},inplace=True)
    
    t_statistic, p_value = stats.ttest_rel(out_store['before_restrict'],out_store['after_restrict'])
    
    
    print("t-statistic:", t_statistic)
    print("p-value:", p_value)

    alpha = 0.05
    if p_value < alpha:
        print("귀무가설을 기각합니다. 두 샘플의 평균은 통계적으로 유의미하게 다릅니다.")
    else:
        print("귀무가설을 기각하지 않습니다. 두 샘플의 평균은 통계적으로 차이가 없습니다.")
        
    print("규제 전 사용금액 총합: " + str(out_store['before_restrict'].sum()))
    print("규제 후 사용금액 총합: " + str(out_store['after_restrict'].sum()))
    print((out_store['after_restrict'].sum()-out_store['before_restrict'].sum())/out_store['before_restrict'].sum()*100)

    #out_store['증감율'] = (out_store['after_restrict']-out_store['before_restrict'])/out_store['before_restrict']*100
    #out_store = out_store.sort_values(by='증감율',ascending=False)
    print('----------------------------------\n\n')

음식_//_후라이드/양념치킨
t-statistic: -6.144919077083443
p-value: 3.257969162350648e-07
귀무가설을 기각합니다. 두 샘플의 평균은 통계적으로 유의미하게 다릅니다.
규제 전 사용금액 총합: 32960507
규제 후 사용금액 총합: 47631450
44.51067151363903
----------------------------------


음식_//_냉면집
t-statistic: -1.783030303030303
p-value: 0.32539534440073603
귀무가설을 기각하지 않습니다. 두 샘플의 평균은 통계적으로 차이가 없습니다.
규제 전 사용금액 총합: 8489000
규제 후 사용금액 총합: 11431000
34.65661444221934
----------------------------------


음식_//_커피전문점/카페/다방
t-statistic: -2.0531086924222444
p-value: 0.04918012001499699
귀무가설을 기각합니다. 두 샘플의 평균은 통계적으로 유의미하게 다릅니다.
규제 전 사용금액 총합: 21057687
규제 후 사용금액 총합: 24053560
14.226980389631585
----------------------------------


음식_//_한식/백반/한정식
t-statistic: -6.215651914711908
p-value: 1.5326360082333242e-09
귀무가설을 기각합니다. 두 샘플의 평균은 통계적으로 유의미하게 다릅니다.
규제 전 사용금액 총합: 305021037
규제 후 사용금액 총합: 362807223
18.944983784839735
----------------------------------


음식_//_죽전문점
t-statistic: nan
p-value: nan
귀무가설을 기각하지 않습니다. 두 샘플의 평균은 통계적으로 차이가 없습니다.
규제 전 사용금액 총합: 0
규제 후 사용금액 총합: 0
n

C:\Users\addmin\AppData\Local\Temp\ipykernel_1344\1581788411.py:29: RuntimeWarning: invalid value encountered in scalar divide
  print((out_store['after_restrict'].sum()-out_store['before_restrict'].sum())/out_store['before_restrict'].sum()*100)


t-statistic: 0.5853784525020188
p-value: 0.5994143590860452
귀무가설을 기각하지 않습니다. 두 샘플의 평균은 통계적으로 차이가 없습니다.
규제 전 사용금액 총합: 4053100
규제 후 사용금액 총합: 3667000
-9.52604179516913
----------------------------------


음식_//_갈비/삼겹살
t-statistic: -2.755675006071029
p-value: 0.020279764266635275
귀무가설을 기각합니다. 두 샘플의 평균은 통계적으로 유의미하게 다릅니다.
규제 전 사용금액 총합: 16263500
규제 후 사용금액 총합: 23573500
44.94727457189412
----------------------------------


음식_//_라면김밥분식
t-statistic: -0.16467828849640562
p-value: 0.8728365111129758
귀무가설을 기각하지 않습니다. 두 샘플의 평균은 통계적으로 차이가 없습니다.
규제 전 사용금액 총합: 4722100
규제 후 사용금액 총합: 4844000
2.5814785794455855
----------------------------------


음식_//_떡볶이전문
t-statistic: nan
p-value: nan
귀무가설을 기각하지 않습니다. 두 샘플의 평균은 통계적으로 차이가 없습니다.
규제 전 사용금액 총합: 0
규제 후 사용금액 총합: 0
nan
----------------------------------


음식_//_종합부페


C:\Users\addmin\AppData\Local\Temp\ipykernel_1344\1581788411.py:29: RuntimeWarning: invalid value encountered in scalar divide
  print((out_store['after_restrict'].sum()-out_store['before_restrict'].sum())/out_store['before_restrict'].sum()*100)


t-statistic: 0.31868131868131866
p-value: 0.8035987922371838
귀무가설을 기각하지 않습니다. 두 샘플의 평균은 통계적으로 차이가 없습니다.
규제 전 사용금액 총합: 2174000
규제 후 사용금액 총합: 2145000
-1.3339466421343145
----------------------------------


음식_//_제과점
t-statistic: -1.1429640209481058
p-value: 0.4575918879158661
귀무가설을 기각하지 않습니다. 두 샘플의 평균은 통계적으로 차이가 없습니다.
규제 전 사용금액 총합: 6260100
규제 후 사용금액 총합: 7940600
26.844619095541606
----------------------------------


음식_//_정통양식/경양식
t-statistic: 0.934713266744708
p-value: 0.4486104484240869
귀무가설을 기각하지 않습니다. 두 샘플의 평균은 통계적으로 차이가 없습니다.
규제 전 사용금액 총합: 4129600
규제 후 사용금액 총합: 2792600
-32.37601704765595
----------------------------------


음식_//_국수/만두/칼국수
t-statistic: -3.0011630376709313
p-value: 0.012054755367633174
귀무가설을 기각합니다. 두 샘플의 평균은 통계적으로 유의미하게 다릅니다.
규제 전 사용금액 총합: 11789960
규제 후 사용금액 총합: 16176500
37.20572419244849
----------------------------------


음식_//_기타고기요리
t-statistic: 0.014621952417025085
p-value: 0.9887418051273884
귀무가설을 기각하지 않습니다. 두 샘플의 평균은 통계적으로 차이가 없습니다.
규제 전 사용금액 총합: 13687800
규제

...

t-statistic: nan
p-value: nan
귀무가설을 기각하지 않습니다. 두 샘플의 평균은 통계적으로 차이가 없습니다.
규제 전 사용금액 총합: 0
규제 후 사용금액 총합: 0
nan
----------------------------------


음식_//_기타전문서양음식점


C:\Users\addmin\AppData\Local\Temp\ipykernel_1344\1581788411.py:29: RuntimeWarning: invalid value encountered in scalar divide
  print((out_store['after_restrict'].sum()-out_store['before_restrict'].sum())/out_store['before_restrict'].sum()*100)


t-statistic: nan
p-value: nan
귀무가설을 기각하지 않습니다. 두 샘플의 평균은 통계적으로 차이가 없습니다.
규제 전 사용금액 총합: 0
규제 후 사용금액 총합: 0
nan
----------------------------------


음식_//_국도휴게소


C:\Users\addmin\AppData\Local\Temp\ipykernel_1344\1581788411.py:29: RuntimeWarning: invalid value encountered in scalar divide
  print((out_store['after_restrict'].sum()-out_store['before_restrict'].sum())/out_store['before_restrict'].sum()*100)


ValueError: cannot set a frame with no defined index and a scalar

### 대형마트의 가맹점 주소별 T-Test

In [68]:
for place in data_가맹_부여_기타제거['가맹점 주소'].unique():
    print(place)
    
    tmp_df = data_가맹_부여_기타제거.loc[(data_가맹_부여_기타제거['가맹점 주소'] == place)]
    grp_side_store = tmp_df.groupby(['연도','월','연령대','읍/면','골목상권분류','소분류_pp','가맹점 주소','가맹점 사업자 번호']).agg({'사용금액':'sum', '사용자코드':'nunique'}).reset_index()
    before_side_store = grp_side_store[(grp_side_store['연도'] == '2023년') & (grp_side_store['월'] == '6월') & (grp_side_store['골목상권분류']=='대형')].groupby('가맹점 사업자 번호')['사용금액'].sum().reset_index()
    after_side_store = grp_side_store[(grp_side_store['연도'] == '2023년') & (grp_side_store['월'] == '7월') & (grp_side_store['골목상권분류']=='대형')].groupby('가맹점 사업자 번호')['사용금액'].sum().reset_index()
    
    before_side_store = outlier(before_side_store, '사용금액')
    after_side_store = outlier(after_side_store, '사용금액')
    out_store = pd.merge(before_side_store,after_side_store,how='inner',on='가맹점 사업자 번호')
    out_store = out_store.loc[(out_store['이상치_사용금액_x']=='정상') & (out_store['이상치_사용금액_y']=='정상')].drop(columns=['이상치_사용금액_x','이상치_사용금액_y'])
    out_store.rename(columns={'사용금액_x':'before_restrict', '사용금액_y':'after_restrict'},inplace=True)
    
    t_statistic, p_value = stats.ttest_rel(out_store['before_restrict'],out_store['after_restrict'])
    
    
    print("t-statistic:", t_statistic)
    print("p-value:", p_value)

    alpha = 0.05
    if p_value < alpha:
        print("귀무가설을 기각합니다. 두 샘플의 평균은 통계적으로 유의미하게 다릅니다.")
    else:
        print("귀무가설을 기각하지 않습니다. 두 샘플의 평균은 통계적으로 차이가 없습니다.")
        
    print("규제 전 사용금액 총합: " + str(out_store['before_restrict'].sum()))
    print("규제 후 사용금액 총합: " + str(out_store['after_restrict'].sum()))
    print((out_store['after_restrict'].sum()-out_store['before_restrict'].sum())/out_store['before_restrict'].sum()*100)

    #out_store['증감율'] = (out_store['after_restrict']-out_store['before_restrict'])/out_store['before_restrict']*100
    #out_store = out_store.sort_values(by='증감율',ascending=False)
    print('----------------------------------\n\n')

부여읍
t-statistic: -1.5259698288142354
p-value: 0.13401582116406238
귀무가설을 기각하지 않습니다. 두 샘플의 평균은 통계적으로 차이가 없습니다.
규제 전 사용금액 총합: 86509864
규제 후 사용금액 총합: 108066782
24.918450917920758
----------------------------------


규암면
t-statistic: -1.467471039744539
p-value: 0.15579045895353033
귀무가설을 기각하지 않습니다. 두 샘플의 평균은 통계적으로 차이가 없습니다.
규제 전 사용금액 총합: 31365524
규제 후 사용금액 총합: 40092667
27.823998731856037
----------------------------------


기타
t-statistic: 4.280591958401233
p-value: 6.263815868430999e-05
귀무가설을 기각합니다. 두 샘플의 평균은 통계적으로 유의미하게 다릅니다.
규제 전 사용금액 총합: 360759291
규제 후 사용금액 총합: 137112391
-61.993386055301904
----------------------------------


홍산면
t-statistic: -1.375790370004505
p-value: 0.2021531316218482
귀무가설을 기각하지 않습니다. 두 샘플의 평균은 통계적으로 차이가 없습니다.
규제 전 사용금액 총합: 12450287
규제 후 사용금액 총합: 16634934
33.61084768567986
----------------------------------




### 대형마트의 업종 대분류별 T-Test

In [69]:
for category in data_가맹_부여_기타제거.loc[(data_가맹_부여_기타제거['연도']=='2023년') & ((data_가맹_부여_기타제거['월']=='6월') | (data_가맹_부여_기타제거['월']=='7월')) & (data_가맹_부여_기타제거['골목상권분류'] == '대형'),'대분류'].unique():
    print(category)
    
    tmp_df = data_가맹_부여_기타제거.loc[(data_가맹_부여_기타제거['대분류'] == category)]
    grp_side_store = tmp_df.groupby(['연도','월','연령대','읍/면','골목상권분류','가맹점 주소','가맹점 사업자 번호']).agg({'사용금액':'sum', '사용자코드':'nunique'}).reset_index()
    before_side_store = grp_side_store[(grp_side_store['연도'] == '2023년') & (grp_side_store['월'] == '6월') & (grp_side_store['골목상권분류']=='대형')].groupby('가맹점 사업자 번호')['사용금액'].sum().reset_index()
    after_side_store = grp_side_store[(grp_side_store['연도'] == '2023년') & (grp_side_store['월'] == '7월') & (grp_side_store['골목상권분류']=='대형')].groupby('가맹점 사업자 번호')['사용금액'].sum().reset_index()
    
    before_side_store = outlier(before_side_store, '사용금액')
    after_side_store = outlier(after_side_store, '사용금액')
    out_store = pd.merge(before_side_store,after_side_store,how='inner',on='가맹점 사업자 번호')
    out_store = out_store.loc[(out_store['이상치_사용금액_x']=='정상') & (out_store['이상치_사용금액_y']=='정상')].drop(columns=['이상치_사용금액_x','이상치_사용금액_y'])
    out_store.rename(columns={'사용금액_x':'before_restrict', '사용금액_y':'after_restrict'},inplace=True)
    
    t_statistic, p_value = stats.ttest_rel(out_store['before_restrict'],out_store['after_restrict'])
    
    
    print("t-statistic:", t_statistic)
    print("p-value:", p_value)

    alpha = 0.05
    if p_value < alpha:
        print("귀무가설을 기각합니다. 두 샘플의 평균은 통계적으로 유의미하게 다릅니다.")
    else:
        print("귀무가설을 기각하지 않습니다. 두 샘플의 평균은 통계적으로 차이가 없습니다.")
    
    print("규제 전 사용금액 총합: " + str(out_store['before_restrict'].sum()))
    print("규제 후 사용금액 총합: " + str(out_store['after_restrict'].sum()))
    print((out_store['after_restrict'].sum()-out_store['before_restrict'].sum())/out_store['before_restrict'].sum()*100)

    #out_store['증감율'] = (out_store['after_restrict']-out_store['before_restrict'])/out_store['before_restrict']*100
    #out_store = out_store.sort_values(by='증감율',ascending=False)
    print('----------------------------------\n\n')

대형마트
t-statistic: 5.4239292316880805
p-value: 2.8426941112494315e-06
귀무가설을 기각합니다. 두 샘플의 평균은 통계적으로 유의미하게 다릅니다.
규제 전 사용금액 총합: 542421659
규제 후 사용금액 총합: 102330293
-81.13454886948016
----------------------------------


자동차
t-statistic: -1.2495791838948693
p-value: 0.21488315855683307
귀무가설을 기각하지 않습니다. 두 샘플의 평균은 통계적으로 차이가 없습니다.
규제 전 사용금액 총합: 144559479
규제 후 사용금액 총합: 169052819
16.943434058724023
----------------------------------


농업관련
t-statistic: 1.2111097529970214
p-value: 0.24151502577413636
귀무가설을 기각하지 않습니다. 두 샘플의 평균은 통계적으로 차이가 없습니다.
규제 전 사용금액 총합: 17286100
규제 후 사용금액 총합: 14934850
-13.601969212257249
----------------------------------




### 연령대별 골목상권 소비에 대한 T-Test

In [70]:
for age in data_가맹_부여_기타제거['연령대'].unique():
    print(age)
    
    tmp_df = data_가맹_부여_기타제거.loc[(data_가맹_부여_기타제거['연령대'] == age)]
    grp_side_store = tmp_df.groupby(['연도','월','사용자코드','연령대','읍/면','골목상권분류','대분류','소분류_pp','가맹점 주소']).agg({'사용금액':'sum'}).reset_index()
    before_side_store = grp_side_store[(grp_side_store['연도'] == '2023년') & (grp_side_store['월'] == '6월') & (grp_side_store['골목상권분류']=='골목상권')].groupby('사용자코드')['사용금액'].sum().reset_index()
    after_side_store = grp_side_store[(grp_side_store['연도'] == '2023년') & (grp_side_store['월'] == '7월') & (grp_side_store['골목상권분류']=='골목상권')].groupby('사용자코드')['사용금액'].sum().reset_index()
    
    before_side_store = outlier(before_side_store, '사용금액')
    after_side_store = outlier(after_side_store, '사용금액')
    out_store = pd.merge(before_side_store,after_side_store,how='inner',on='사용자코드')
    out_store = out_store.loc[(out_store['이상치_사용금액_x']=='정상') & (out_store['이상치_사용금액_y']=='정상')].drop(columns=['이상치_사용금액_x','이상치_사용금액_y'])
    out_store.rename(columns={'사용금액_x':'before_restrict', '사용금액_y':'after_restrict'},inplace=True)
    
    t_statistic, p_value = stats.ttest_rel(out_store['before_restrict'],out_store['after_restrict'])
    
    
    print("t-statistic:", t_statistic)
    print("p-value:", p_value)

    alpha = 0.05
    if p_value < alpha:
        print("귀무가설을 기각합니다. 두 샘플의 평균은 통계적으로 유의미하게 다릅니다.")
    else:
        print("귀무가설을 기각하지 않습니다. 두 샘플의 평균은 통계적으로 차이가 없습니다.")
        
    print("규제 전 사용금액 총합: " + str(out_store['before_restrict'].sum()))
    print("규제 후 사용금액 총합: " + str(out_store['after_restrict'].sum()))
    print((out_store['after_restrict'].sum()-out_store['before_restrict'].sum())/out_store['before_restrict'].sum()*100)

    #out_store['증감율'] = (out_store['after_restrict']-out_store['before_restrict'])/out_store['before_restrict']*100
    #out_store = out_store.sort_values(by='증감율',ascending=False)
    print('----------------------------------\n\n')

30대
t-statistic: -5.769388280751982
p-value: 1.2505813605387295e-08
귀무가설을 기각합니다. 두 샘플의 평균은 통계적으로 유의미하게 다릅니다.
규제 전 사용금액 총합: 51912510
규제 후 사용금액 총합: 65091240
25.386424197173284
----------------------------------


40대
t-statistic: -7.307815189087922
p-value: 4.878107749553847e-13
귀무가설을 기각합니다. 두 샘플의 평균은 통계적으로 유의미하게 다릅니다.
규제 전 사용금액 총합: 127987984
규제 후 사용금액 총합: 155128701
21.205675839069393
----------------------------------


70대이상
t-statistic: -3.2392920199228707
p-value: 0.0012313662472621704
귀무가설을 기각합니다. 두 샘플의 평균은 통계적으로 유의미하게 다릅니다.
규제 전 사용금액 총합: 97360723
규제 후 사용금액 총합: 106537782
9.425832838155895
----------------------------------


50대
t-statistic: -8.330293834091497
p-value: 1.5848805289654738e-16
귀무가설을 기각합니다. 두 샘플의 평균은 통계적으로 유의미하게 다릅니다.
규제 전 사용금액 총합: 208769297
규제 후 사용금액 총합: 251330815
20.386866561130397
----------------------------------


60대
t-statistic: -5.140882870511395
p-value: 3.0027334860830957e-07
귀무가설을 기각합니다. 두 샘플의 평균은 통계적으로 유의미하게 다릅니다.
규제 전 사용금액 총합: 244663592
규제 후 사용금액 총합: 2733

### 사용자 거주지별 골목상권 소비에 대한 T-Test

In [71]:
for place in data_가맹_부여_기타제거['읍/면'].unique():
    print(place)
    
    tmp_df = data_가맹_부여_기타제거.loc[(data_가맹_부여_기타제거['읍/면'] == place)]
    grp_side_store = tmp_df.groupby(['연도','월','사용자코드','연령대','읍/면','골목상권분류','대분류','소분류_pp','가맹점 주소']).agg({'사용금액':'sum'}).reset_index()
    before_side_store = grp_side_store[(grp_side_store['연도'] == '2023년') & (grp_side_store['월'] == '6월') & (grp_side_store['골목상권분류']=='골목상권')].groupby('사용자코드')['사용금액'].sum().reset_index()
    after_side_store = grp_side_store[(grp_side_store['연도'] == '2023년') & (grp_side_store['월'] == '7월') & (grp_side_store['골목상권분류']=='골목상권')].groupby('사용자코드')['사용금액'].sum().reset_index()
    
    before_side_store = outlier(before_side_store, '사용금액')
    after_side_store = outlier(after_side_store, '사용금액')
    out_store = pd.merge(before_side_store,after_side_store,how='inner',on='사용자코드')
    out_store = out_store.loc[(out_store['이상치_사용금액_x']=='정상') & (out_store['이상치_사용금액_y']=='정상')].drop(columns=['이상치_사용금액_x','이상치_사용금액_y'])
    out_store.rename(columns={'사용금액_x':'before_restrict', '사용금액_y':'after_restrict'},inplace=True)
    
    t_statistic, p_value = stats.ttest_rel(out_store['before_restrict'],out_store['after_restrict'])
    
    
    print("t-statistic:", t_statistic)
    print("p-value:", p_value)

    alpha = 0.05
    if p_value < alpha:
        print("귀무가설을 기각합니다. 두 샘플의 평균은 통계적으로 유의미하게 다릅니다.")
    else:
        print("귀무가설을 기각하지 않습니다. 두 샘플의 평균은 통계적으로 차이가 없습니다.")
        
    print("규제 전 사용금액 총합: " + str(out_store['before_restrict'].sum()))
    print("규제 후 사용금액 총합: " + str(out_store['after_restrict'].sum()))
    print((out_store['after_restrict'].sum()-out_store['before_restrict'].sum())/out_store['before_restrict'].sum()*100)

    #out_store['증감율'] = (out_store['after_restrict']-out_store['before_restrict'])/out_store['before_restrict']*100
    #out_store = out_store.sort_values(by='증감율',ascending=False)
    print('----------------------------------\n\n')

부여읍
t-statistic: -13.061723360381812
p-value: 5.425869421537197e-38
귀무가설을 기각합니다. 두 샘플의 평균은 통계적으로 유의미하게 다릅니다.
규제 전 사용금액 총합: 327500309
규제 후 사용금액 총합: 408782251
24.8188901708792
----------------------------------


기타
t-statistic: -3.183297352960842
p-value: 0.0014774801363786417
귀무가설을 기각합니다. 두 샘플의 평균은 통계적으로 유의미하게 다릅니다.
규제 전 사용금액 총합: 195668797
규제 후 사용금액 총합: 209882675
7.264253789018799
----------------------------------


규암면
t-statistic: -6.105974901779682
p-value: 1.2763961901368172e-09
귀무가설을 기각합니다. 두 샘플의 평균은 통계적으로 유의미하게 다릅니다.
규제 전 사용금액 총합: 164119161
규제 후 사용금액 총합: 189514159
15.473511956352251
----------------------------------


홍산면
t-statistic: -3.1150163403523092
p-value: 0.001975322935872303
귀무가설을 기각합니다. 두 샘플의 평균은 통계적으로 유의미하게 다릅니다.
규제 전 사용금액 총합: 38959490
규제 후 사용금액 총합: 44939100
15.348275862954058
----------------------------------




In [72]:
for place in data_가맹_부여_기타제거['읍/면'].unique():
    for age in data_가맹_부여_기타제거['연령대'].unique():
        
        print(place)
        print(age)
        tmp_df = data_가맹_부여_기타제거.loc[(data_가맹_부여_기타제거['읍/면'] == place) & (data_가맹_부여_기타제거['연령대'] == age)]
        grp_side_store = tmp_df.groupby(['연도','월','사용자코드','연령대','읍/면','골목상권분류','대분류','소분류_pp','가맹점 주소']).agg({'사용금액':'sum'}).reset_index()
        before_side_store = grp_side_store[(grp_side_store['연도'] == '2023년') & (grp_side_store['월'] == '6월') & (grp_side_store['골목상권분류']=='골목상권')].groupby('사용자코드')['사용금액'].sum().reset_index()
        after_side_store = grp_side_store[(grp_side_store['연도'] == '2023년') & (grp_side_store['월'] == '7월') & (grp_side_store['골목상권분류']=='골목상권')].groupby('사용자코드')['사용금액'].sum().reset_index()
        
        before_side_store = outlier(before_side_store, '사용금액')
        after_side_store = outlier(after_side_store, '사용금액')
        out_store = pd.merge(before_side_store,after_side_store,how='inner',on='사용자코드')
        out_store = out_store.loc[(out_store['이상치_사용금액_x']=='정상') & (out_store['이상치_사용금액_y']=='정상')].drop(columns=['이상치_사용금액_x','이상치_사용금액_y'])
        out_store.rename(columns={'사용금액_x':'before_restrict', '사용금액_y':'after_restrict'},inplace=True)
        
        t_statistic, p_value = stats.ttest_rel(out_store['before_restrict'],out_store['after_restrict'])
        
        
        print("t-statistic:", t_statistic)
        print("p-value:", p_value)

        alpha = 0.05
        if p_value < alpha:
            print("귀무가설을 기각합니다. 두 샘플의 평균은 통계적으로 유의미하게 다릅니다.")
        else:
            print("귀무가설을 기각하지 않습니다. 두 샘플의 평균은 통계적으로 차이가 없습니다.")
            
        print("규제 전 사용금액 총합: " + str(out_store['before_restrict'].sum()))
        print("규제 후 사용금액 총합: " + str(out_store['after_restrict'].sum()))
        print((out_store['after_restrict'].sum()-out_store['before_restrict'].sum())/out_store['before_restrict'].sum()*100)

        #out_store['증감율'] = (out_store['after_restrict']-out_store['before_restrict'])/out_store['before_restrict']*100
        #out_store = out_store.sort_values(by='증감율',ascending=False)
        print('----------------------------------\n\n')

부여읍
30대
t-statistic: -4.935491950381413
p-value: 1.2779273753930186e-06
귀무가설을 기각합니다. 두 샘플의 평균은 통계적으로 유의미하게 다릅니다.
규제 전 사용금액 총합: 25837510
규제 후 사용금액 총합: 34128340
32.08834752265214
----------------------------------


부여읍
40대
t-statistic: -5.444488564187578
p-value: 7.863145371982834e-08
귀무가설을 기각합니다. 두 샘플의 평균은 통계적으로 유의미하게 다릅니다.
규제 전 사용금액 총합: 61096214
규제 후 사용금액 총합: 75587289
23.718450049949087
----------------------------------


부여읍
70대이상
t-statistic: -3.9676381100790445
p-value: 8.761794695245427e-05
귀무가설을 기각합니다. 두 샘플의 평균은 통계적으로 유의미하게 다릅니다.
규제 전 사용금액 총합: 33189986
규제 후 사용금액 총합: 40517068
22.076182858287435
----------------------------------


부여읍
50대
t-statistic: -7.409554283433854
p-value: 3.064807452803681e-13
귀무가설을 기각합니다. 두 샘플의 평균은 통계적으로 유의미하게 다릅니다.
규제 전 사용금액 총합: 102991027
규제 후 사용금액 총합: 130710108
26.914073786253244
----------------------------------


부여읍
60대
t-statistic: -6.047575761633073
p-value: 2.27951657503724e-09
귀무가설을 기각합니다. 두 샘플의 평균은 통계적으로 유의미하게 다릅니다.
규제 전 사용금액 총합: 101825331
규제 후

## 3) 충전 데이터 현황분석

In [ ]:
#충전데이터 불러오기
data_ch_new1 = pd.read_excel('../Data/20231208_사용자충전데이터.xlsx',  sheet_name= 'Result 1')
data_ch_new2 = pd.read_excel('../Data/20231208_사용자충전데이터.xlsx',  sheet_name= 'Result 2')
data_ch_new = pd.concat([data_ch_new1,data_ch_new2],axis=0)

In [4]:
data_ch_new['기준연도'] = pd.to_datetime(data_ch_new['month'],format='%Y%m').dt.strftime('%Y')

In [5]:
#1. 정책 발행이 줄어들었음
data_ch_new.groupby('기준연도')['발행(정책+민간)'].sum().reset_index().iloc[2:,:]

,기준연도,발행(정책+민간)
2,2021,40619568250
3,2022,17158428610
4,2023,14230386251


In [6]:
data_ch_new.loc[(data_ch_new['month']>=202101) & (data_ch_new['month']<=202212),'집단구분']='2021_2022년'
data_ch_new.loc[(data_ch_new['month']>202212) & (data_ch_new['month']<=202306),'집단구분']='2023년상반기'
data_ch_new.loc[(data_ch_new['month']>202306) & (data_ch_new['month']<=202310),'집단구분']='2023년하반기'
data_ch_new = data_ch_new.loc[data_ch_new['집단구분'] != 'nan']

In [13]:
len(data_ch_new.loc[data_ch_new['집단구분']=='2021_2022년','month'].unique())

24

In [11]:
len(data_ch_new.loc[data_ch_new['집단구분']=='2023년상반기','month'].unique())

6

In [12]:
len(data_ch_new.loc[data_ch_new['집단구분']=='2023년하반기','month'].unique())

4

In [16]:
data_ch_new.groupby('집단구분')['month'].nunique()

집단구분
2021_2022년    24
2023년상반기       6
2023년하반기       4
Name: month, dtype: int64

In [20]:
analysis_ch = data_ch_new.groupby('집단구분').agg({'충전금액':'sum','month':'nunique'}).reset_index()

In [27]:
analysis_ch['월 평균'] = analysis_ch['충전금액']/analysis_ch['month']
analysis_ch

,집단구분,충전금액,month,월 평균
0,2021_2022년,144156134441,24,6.006506e+09
1,2023년상반기,32190235023,6,5.365039e+09
2,2023년하반기,23737575631,4,5.934394e+09
